In [1]:
# Perform imports here:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pytz import timezone
from sqlalchemy import create_engine
import math
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import gc
from joblib import Parallel, delayed
import joblib

In [4]:
#df = pd.read_pickle("OLI_SELL.pk")
df = pd.read_pickle("data/GSS0713.pk")
#df = pd.read_pickle("HMA_SELL.pk")
df.head(1)

,SELL_AMOUNT,STOCK_AMOUNT,REVISE_STOCK_AMOUNT,STOCK_ID,ITEM_ID,UPT_DT,COLLECT_DAY,UPT_ID,REG_DT,ITEM_NUM,GOODS_NAME,COLLECT_URL,NORMAL_PRICE_SITE,BRAND_NAME,SITE_NO,CATE_NAMES,BRAND,SELL
0,0,726,726,16437651001,115022,2018-07-13 13:36:16,2018-06-01,FILTER ALGO,2018-06-01,16437651,[바비리스] 1600W 이온 헤어드라이기 접이식/2단/쿨샷 5383K +,http://www.gsshop.com/prd/prd.gs?prdid=16437651,27800,바비리스,1,바디/헤어>헤어케어,init,0


In [3]:
df.SELL_AMOUNT.value_counts().reset_index().sort_values("index",ascending=False)

,index,SELL_AMOUNT
136,499.0,29
194,498.0,11
243,497.0,6
250,496.0,5
229,495.0,7
176,494.0,15
188,493.0,12
259,492.0,4
238,491.0,6
248,490.0,5


In [ ]:
df.SELL_AMOUNT.value_counts().reset_index().sort_values("index",ascending=False)[25:]

In [ ]:
df.SELL_AMOUNT.value_counts().reset_index().sort_values("index",ascending=False)[50:]

In [ ]:
df.SELL_AMOUNT.max()

In [ ]:
n = 500
out = df[df.SELL_AMOUNT>n].STOCK_ID.unique()

In [ ]:
df[df.SELL_AMOUNT>n].groupby("BRAND").size()

In [ ]:
df[df.SELL_AMOUNT>n].groupby("BRAND").ITEM_ID.nunique()

In [ ]:
df[df.SELL_AMOUNT>n].groupby("BRAND").STOCK_ID.nunique()

In [ ]:
df[(df.STOCK_ID.isin(out))&(df.BRAND=="AP")].sort_values(["ITEM_ID", "STOCK_ID", "COLLECT_DAY"])

In [ ]:
df[(df.STOCK_ID.isin(out))&(df.BRAND=="LG")].sort_values(["ITEM_ID", "STOCK_ID", "COLLECT_DAY"])

In [ ]:
df[(df.STOCK_ID.isin(out))&(df.BRAND=="LOREAL")].sort_values(["ITEM_ID", "STOCK_ID", "COLLECT_DAY"])

In [ ]:
df[(df.STOCK_ID.isin(out))&(df.BRAND=="LVMH")].sort_values(["ITEM_ID", "STOCK_ID", "COLLECT_DAY"])

## 루이비똥 봐보자 ...

In [ ]:
n

In [ ]:
lvmh = df[(df.SELL_AMOUNT<n)&(df.BRAND=="LVMH")]

In [ ]:
lvmh.SELL.sort_values(ascending=False)

In [ ]:
lvmh.loc[lvmh.SELL.sort_values(ascending=False).index[1:100]].SELL_AMOUNT

In [ ]:
lvmh[lvmh.STOCK_ID==lvmh.loc[504031].STOCK_ID]

In [ ]:
lvmh[(lvmh.SELL_AMOUNT < 480)|(lvmh.SELL_AMOUNT > 500)].SELL.sum()

In [ ]:
temp = lvmh.SELL_AMOUNT.value_counts().sort_values(ascending=False)
temp2 = temp[temp>1].reset_index()
shit = temp2[temp2["index"]>100]['index']

In [ ]:
lvmh[-lvmh.SELL_AMOUNT.isin(shit)].SELL.sum()

## 브랜드별 / 전체 매출액

In [35]:
df=pd.read_pickle('data/OLI0713.pk')
df[df.SELL!=0][['ITEM_ID','SELL','BRAND','SELL_AMOUNT']].head(2)

,ITEM_ID,SELL,BRAND,SELL_AMOUNT
1,8758383,69000,LVMH,1
2,8758383,69000,LVMH,1


In [36]:
pd.DataFrame(df.groupby('BRAND').SELL.sum().astype(str)).T

BRAND,AP,LG,LOREAL,LVMH,init
SELL,193578900.0,521160650.0,428640100.0,98265900.0,10523470450.0


In [34]:
df[df.SELL_AMOUNT<500].to_pickle('data/OLI0713.pk')

In [33]:
df.SELL_AMOUNT.sort_values(ascending=False)

130414    99972
175734    99970
56968     99966
186151    99957
221558    99956
234197    99953
55843     99905
260492    99895
144412    99814
138056    99813
19876     99783
55271     99748
258667    99747
214133    99721
177764    99715
98420     99694
213498    99686
119510    99666
214108    99622
25210     99533
222942    99449
209962    99393
261132     6604
148824     3671
148823     1454
27124      1288
49868      1080
167743     1065
245445      951
202163      944
          ...  
263334     None
263335     None
263349     None
263350     None
263357     None
263361     None
263363     None
263376     None
263378     None
263386     None
263391     None
263392     None
263402     None
263415     None
263434     None
263444     None
263464     None
263471     None
263481     None
263499     None
263662     None
263691     None
263723     None
263727     None
263749     None
263753     None
263777     None
263796     None
263804     None
263807     None
Name: SELL_AMOUNT, Lengt

## 판매량 계산

In [5]:
df.groupby("BRAND").SELL.sum().reset_index()

,BRAND,SELL
0,AP,4.869031e+09
1,LG,4.793041e+09
2,LOREAL,1.969938e+09
3,LVMH,7.868710e+08
4,init,4.364687e+10


In [6]:
df[df.SELL_AMOUNT<500].groupby("BRAND").SELL.sum().reset_index()

,BRAND,SELL
0,AP,4.869031e+09
1,LG,4.793041e+09
2,LOREAL,1.969938e+09
3,LVMH,7.868710e+08
4,init,4.364687e+10


In [ ]:
df[df.SELL_AMOUNT<100].groupby("BRAND").SELL.sum().reset_index()

## 일단 정리하자~~

In [17]:
OLI = pd.read_pickle("OLI_SELL.pk")
GSS = pd.read_pickle("GSS_SELL.pk")
HMA = pd.read_pickle("HMA_SELL.pk")

In [18]:
#OLI[OLI.SELL_AMOUNT<500]
HMA = HMA[HMA.SELL_AMOUNT<500]
GSS = GSS[GSS.SELL_AMOUNT<500]

In [19]:
n = 500
lvmh = GSS[(GSS.SELL_AMOUNT<n)&(GSS.BRAND=="LVMH")]
temp = lvmh.SELL_AMOUNT.value_counts().sort_values(ascending=False)
temp2 = temp[temp>1].reset_index()
shit = temp2[temp2["index"]>100]['index']

In [20]:
GSS[-((GSS.BRAND=="LVMH")&(GSS.SELL_AMOUNT.isin(shit)))].groupby("BRAND").SELL.sum()

BRAND
AP        4.869031e+09
LG        4.793041e+09
LOREAL    1.969938e+09
LVMH      7.868710e+08
init      4.364687e+10
Name: SELL, dtype: float64

In [21]:
HMA.groupby("BRAND").SELL.sum()

BRAND
AP        2.055956e+09
LG        1.048695e+09
LOREAL    2.701683e+09
LVMH      4.114830e+08
init      7.396535e+09
Name: SELL, dtype: float64

In [22]:
OLI.groupby("BRAND").SELL.sum()

BRAND
AP        1.935789e+08
LG        5.211606e+08
LOREAL    4.382521e+08
LVMH      9.826590e+07
init      1.145958e+11
Name: SELL, dtype: float64

In [23]:
GSS=GSS[-((GSS.BRAND=="LVMH")&(GSS.SELL_AMOUNT.isin(shit)))]

In [24]:
OLI.to_pickle("OLI0713.pk")
HMA.to_pickle("HMA0713.pk")
GSS.to_pickle("GSS0713.pk")

## 정리한 거 계산 ..

In [26]:
OLI_pivot = OLI.groupby(["BRAND", "CATE_NAMES"]).SELL.sum().reset_index().pivot(index="CATE_NAMES", columns="BRAND", values="SELL")
HMA_pivot = HMA.groupby(["BRAND", "CATE_NAMES"]).SELL.sum().reset_index().pivot(index="CATE_NAMES", columns="BRAND", values="SELL")
GSS_pivot = GSS.groupby(["BRAND", "CATE_NAMES"]).SELL.sum().reset_index().pivot(index="CATE_NAMES", columns="BRAND", values="SELL")

In [27]:
OLI_pivot["SITE_NAME"] = "Oliveyoung"
GSS_pivot["SITE_NAME"] = "GSSHOP"
HMA_pivot["SITE_NAME"] = "HMALL"

In [31]:
report = pd.concat([OLI_pivot, GSS_pivot, HMA_pivot]).reset_index()
report = report.sort_values(["CATE_NAMES", "SITE_NAME"]).reset_index(drop=True)
report[["CATE_NAMES", "SITE_NAME", "AP", "LG", "LOREAL", "LVMH"]].to_csv("REPORT0713.csv", index=False)
report[["AP", "LG", "LOREAL", "LVMH", "SITE_NAME"]].groupby("SITE_NAME").sum().to_csv("REPORT20713.csv")

## 연습장

In [73]:
old[(old.ITEM_ID==8764787)&(old.STOCK_ID=='-1251111349027131874')]

,SELL_AMOUNT,STOCK_AMOUNT,REVISE_STOCK_AMOUNT,STOCK_ID,ITEM_ID,UPT_DT,COLLECT_DAY,UPT_ID,REG_DT,ITEM_NUM,NORMAL_PRICE_SITE,BRAND_NAME,SITE_NO,CATE_NAMES,BRAND,SELL
46422,0,210,210,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-02,FILTER ALGO,2018-06-02,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,0
46423,None,106,106,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-03,FILTER ALGO,2018-06-03,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,NaN
46424,None,0,0,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-04,FILTER ALGO,2018-06-04,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,NaN
46425,0,0,0,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-05,FILTER ALGO,2018-06-05,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,0
46426,0,0,0,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-06,FILTER ALGO,2018-06-06,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,0
46427,None,59,None,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-07,FILTER ALGO,2018-06-07,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,NaN
46428,None,37,37,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-08,FILTER ALGO,2018-06-08,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,NaN
46429,None,26,26,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-09,FILTER ALGO,2018-06-09,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,NaN
46430,None,14,14,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-10,FILTER ALGO,2018-06-10,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,NaN
46431,None,141,None,-1251111349027131874,8764787,2018-07-11 17:05:59,2018-06-11,FILTER ALGO,2018-06-11,A000000112419,19000,비욘드,4,스킨케어>베이직 기초,LG,NaN


In [65]:
OLI[(OLI.ITEM_ID==11961831)&(OLI.STOCK_ID=='7405510148804182974')]

,SELL_AMOUNT,STOCK_AMOUNT,REVISE_STOCK_AMOUNT,STOCK_ID,ITEM_ID,UPT_DT,COLLECT_DAY,UPT_ID,REG_DT,ITEM_NUM,GOODS_NAME,COLLECT_URL,NORMAL_PRICE_SITE,BRAND_NAME,SITE_NO,CATE_NAMES,BRAND,SELL
253731,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-02,FILTER ALGO,2018-06-02,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0
253732,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-03,FILTER ALGO,2018-06-03,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0
253733,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-04,FILTER ALGO,2018-06-04,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0
253734,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-05,FILTER ALGO,2018-06-05,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0
253735,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-06,FILTER ALGO,2018-06-06,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0
253736,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-07,FILTER ALGO,2018-06-07,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0
253737,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-08,FILTER ALGO,2018-06-08,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0
253738,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-09,FILTER ALGO,2018-06-09,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0
253739,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-10,FILTER ALGO,2018-06-10,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0
253740,0,99999,99999,7405510148804182974,11961831,2018-07-13 13:09:06,2018-06-11,FILTER ALGO,2018-06-11,B000000114806,포레오 UFO 마스크 디바이스,http://www.oliveyoung.co.kr/store/goods/getGoo...,353000,미분류,4,스킨케어>마스크/팩,init,0


In [79]:
temp = OLI.groupby("ITEM_ID").STOCK_AMOUNT.nunique()
st = temp[temp==1].reset_index()
temp2 = old[old.ITEM_ID.isin(st.ITEM_ID)]
temp3 = temp2[temp2.STOCK_AMOUNT.isin([99,999,9999,99999,999999,9999999])]
temp3.STOCK_AMOUNT.unique()

array([99999.0], dtype=object)

In [85]:
temp = GSS.groupby("ITEM_ID").STOCK_AMOUNT.nunique()
st = temp[temp==1].reset_index()
temp2 = GSS[GSS.ITEM_ID.isin(st.ITEM_ID)]
temp3 = temp2[temp2.STOCK_AMOUNT.isin([99,999,9999,99999,999999,9999999])]
temp3.STOCK_AMOUNT.unique()

array([99.0, 999.0, 9999.0, 99999.0], dtype=object)

In [87]:
temp3.groupby()

(33295, 18)

In [84]:
temp = HMA.groupby("ITEM_ID").STOCK_AMOUNT.nunique()
st = temp[temp==1].reset_index()
temp2 = HMA[HMA.ITEM_ID.isin(st.ITEM_ID)]
temp3 = temp2[temp2.STOCK_AMOUNT.isin([99,999,9999,99999,999999,9999999])]
temp3.STOCK_AMOUNT.unique()

array([999.0, 99.0, 999999.0, 9999.0, 99999.0], dtype=object)

## URL 뽑기

In [119]:
AP = "12819911 2313319 7563091 3856973 12081822 12819947 12819943 12819948 12081825 12803503".split(" ")
LG = "3059399 4044564 3550578 3857804 2030249 4044557 3855427 3858137 2032728 2729103".split(" ")
LOREAL = "12850037 12850038 12808816 1838281 8845990 10365990".split(" ")
LVMH = "12803502 3854124 12795432 1807377 3236579 12803524 1835433 11548531 1852738 12752682".split(" ")

In [120]:
AP = [int(i) for i in AP]
LG = [int(i) for i in LG]
LOREAL = [int(i) for i in LOREAL]
LVMH = [int(i) for i in LVMH]

In [155]:
l1 = HMA[HMA.ITEM_ID.isin(AP)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l2 = HMA[HMA.ITEM_ID.isin(LG)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l3 = HMA[HMA.ITEM_ID.isin(LOREAL)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l4 = HMA[HMA.ITEM_ID.isin(LVMH)][['COLLECT_URL', 'BRAND']].drop_duplicates()

In [150]:
pd.concat([l1,l2,l3,l4]).to_csv("HMA.csv")

In [169]:
AP = "8750156 12721985 12721996 12794053 12722000 12756680 12597033 12609406 1445434 12733865".split(" ")
LG = "12625490 12625344 12180291 12625467 12625354 12625461 12392147 12625393 12625392 12755019".split(" ")
LOREAL = "5312607 5312588 5319995 5312598 5312606 4338237 5319964 4337644 5320003 1760318".split(" ")
LVMH = "12722022 12625534 12625535 12834127 12834129 12625253 6672884 12834128 12834130".split(" ")

In [170]:
AP = [int(i) for i in AP]
LG = [int(i) for i in LG]
LOREAL = [int(i) for i in LOREAL]
LVMH = [int(i) for i in LVMH]

In [171]:
l1 = GSS[GSS.ITEM_ID.isin(AP)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l2 = GSS[GSS.ITEM_ID.isin(LG)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l3 = GSS[GSS.ITEM_ID.isin(LOREAL)][['COLLECT_URL', 'BRAND']].drop_duplicates()
l4 = GSS[GSS.ITEM_ID.isin(LVMH)][['COLLECT_URL', 'BRAND']].drop_duplicates()

In [172]:
pd.concat([l1,l2,l3,l4]).to_csv("GSS.csv")

In [173]:
l2

,COLLECT_URL,BRAND
24892,http://www.gsshop.com/prd/prd.gs?prdid=31693355,LG
73036,http://www.gsshop.com/prd/prd.gs?prdid=31693366,LG
118171,http://www.gsshop.com/deal/deal.gs?dealNo=3138...,LG
121503,http://www.gsshop.com/prd/prd.gs?prdid=31693348,LG
175726,http://www.gsshop.com/prd/prd.gs?prdid=31693344,LG
177656,http://www.gsshop.com/prd/prd.gs?prdid=31690595,LG
209942,http://www.gsshop.com/prd/prd.gs?prdid=31622137,LG
209963,http://www.gsshop.com/prd/prd.gs?prdid=31693346,LG
221434,http://www.gsshop.com/prd/prd.gs?prdid=31693339,LG
266312,http://www.gsshop.com/prd/prd.gs?prdid=31582059,LG


In [167]:
GSS[GSS.ITEM_ID==12819911]

,SELL_AMOUNT,STOCK_AMOUNT,REVISE_STOCK_AMOUNT,STOCK_ID,ITEM_ID,UPT_DT,COLLECT_DAY,UPT_ID,REG_DT,ITEM_NUM,GOODS_NAME,COLLECT_URL,NORMAL_PRICE_SITE,BRAND_NAME,SITE_NO,CATE_NAMES,BRAND,SELL
